In [4]:
#Name: PRISMA2GeoTIFF.py
#Description: reads he5 PRISMA files content and converts it to a GeoTIFF.
#All 66 VNIR bands and 173 SWIR bands are converted in one single GeoTIFF file.
#input is a PRISMA he5 file and output is a GeoTIFF with the same name in the same path
#Author: martin rapilly, mrapilly60@uasd.edu.do/martin.rapilly@get.omp.eu

#import libraries
import h5py
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from itertools import chain
from osgeo import gdal, osr
import os

In [5]:
#np.set_printoptions(threshold=np.inf)#optional: uncommenting this line will show full arrays when printing on the console. Not recommended as he5 PRISMA files contain many values that can overrun memory

def PRISMA2GeoTIFF (filename):
    #open he5 file and read its content
    f = h5py.File(filename,'r')
    def print_name(name, obj):
        if isinstance(obj, h5py.Dataset):
            print ('Dataset:', name)
        elif isinstance(obj, h5py.Group):
            print ('Group:', name)
    with h5py.File(filename, 'r')  as h5f: # file will be closed when we exit from WITH scope
        h5f.visititems(print_name)


        #read SWIR and VNIR cube contents
        SWIRcube = h5f['HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/SWIR_Cube'][()]#[()] is to get the value. Can be replaced with .value
        VNIRcube = h5f['HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/VNIR_Cube'][()]

        #read latitude and longitude contents
        lat = h5f['HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Latitude'][()]
        lon = h5f['HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Longitude'][()]

        #checks SWIR, VNIR and latitude/longitude array shapes
        print ("SWIRcube.shape",SWIRcube.shape)
        print ("VNIRcube.shape",VNIRcube.shape)
        print ("lat.shape",lat.shape)

        #create lists from latitude/longitude values
        lonIter=list(chain.from_iterable(lon))
        latIter=list(chain.from_iterable(lat))




        #create a list from VNIR and SWIR cube values
        listBand=[]
        for band in range(0,VNIRcube.shape[1]):#VNIRcube.shape[1] gives the number of bands (:66)
            for x in range(0,lat.shape[0]):#lat.shape[0] gives the number of rows
                element=VNIRcube[x][band]
                listBand.append(element)
        for band1 in range(0,SWIRcube.shape[1]):#SWIRcube.shape[1] gives the number of bands (:137)
            for x1 in range(0,lat.shape[0]):#lat.shape[0] gives the number of rows
                element=SWIRcube[x1][band1]
                listBand.append(element)

        #convert list with values to a numpy array
        data=np.array(listBand,dtype=np.uint16)

        #checks array shape
        print ("data.shape",data.shape)

        #reshape numpy array with the right number of bands, rows and columns
        dataReshaped=data.reshape([VNIRcube.shape[1]+SWIRcube.shape[1], lat.shape[0], lat.shape[1]])
        print ("reshaped data.shape",dataReshaped.shape)

        #get minimum and maximum latitude and longitude
        xmin,ymin,xmax,ymax = [lon.min(),lat.min(),lon.max(),lat.max()]

        #get pixel spatial resolution
        xres = (xmax-xmin)/lat.shape[1]#lat.shape[1] gives the number of cols
        yres = (ymax-ymin)/lat.shape[0]#lat.shape[0] gives the number of rows

        #define coordinates
        geotransform=(xmin,xres,0,ymax,0, -yres)#zeros (third and fifth parameters) are for rotation

        #define GeoTIFF structure and output filename
        output_raster = gdal.GetDriverByName('GTiff').Create(filename [:-3]+"tif",lat.shape[1], lat.shape[0], VNIRcube.shape[1]+SWIRcube.shape[1] ,gdal.GDT_Float32)  # Open the file

        #loop over all bands and write it to the GeoTIFF
        for b in range(1,VNIRcube.shape[1]+SWIRcube.shape[1]):
            print("converting band",b)
            outband = output_raster.GetRasterBand(b)
            outband.WriteArray(dataReshaped[b,:,:])
        #specify coordinates to WGS84
        output_raster.SetGeoTransform(geotransform)
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)
        output_raster.SetProjection(srs.ExportToWkt())

        #clean memory
        output_raster.FlushCache()
        print("Conversion from he5 PRISMA file to GeoTIFF complete.")

In [9]:
#enter folder path with he5 PRISMA files in it
folderPath= "C:/users/asus/mini_project_2/hdf5"
listImages=[]
for file in os.listdir(folderPath):
      listImages.append(os.path.join(folderPath, file))
print ("he5 image list: ", listImages)

#apply function PRISMA2GeoTIFF
for filename in listImages:
    print("Processing image", filename)
    PRISMA2GeoTIFF(filename)
print ("All files processed.")

he5 image list:  ['C:/users/asus/mini_project_2/hdf5\\raw_file.he5']
Processing image C:/users/asus/mini_project_2/hdf5\raw_file.he5
Group: HDFEOS
Group: HDFEOS/SWATHS
Group: HDFEOS/SWATHS/GCP attributes
Group: HDFEOS/SWATHS/Geocoding attributes
Group: HDFEOS/SWATHS/Geocoding attributes/Ancillary
Group: HDFEOS/SWATHS/PRS_L2D_HCO
Group: HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields
Dataset: HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/SWIR_Cube
Dataset: HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/SWIR_PIXEL_L2_ERR_MATRIX
Dataset: HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/VNIR_Cube
Dataset: HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/VNIR_PIXEL_L2_ERR_MATRIX
Group: HDFEOS/SWATHS/PRS_L2D_HCO/Geocoding Model
Group: HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields
Dataset: HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Latitude
Dataset: HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Longitude
Dataset: HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Time
Group: HDFEOS/SWATHS/PRS_L2D_HCO/Geometric Fields
Dataset: HDFEOS/SWATHS/PRS_L2